첫번째 인간 유전체를 시퀀싱하는데 US\$3 billion가 들었던 것에 비해 오늘날 Illumina의 HiSeq 4000과 같은 genome sequencing machines은 US\$1,000 미만으로 몇시간동안 수천개의 DNA와 RNA 시퀀싱 데이터를 생산해 냄.

생명과학 분야에서의 성공은 이러한 기술에 의해 생산되는 빅 데이터를 적절히 분석하는 능력에 의존을 하며 informatics에서 발전을 채택 하는게 필요하다. Map‐Reduce/Hadoop와 Spark는 우리가 수 시간 내에 gigabytes/ petabytes의 데이터를 계산하고 분석하도록 한다. 예를 들어, Spark는 최근 23분 동안 206개의 기계를 사용하여 100 TB 의 데이터를 정렬하였다.

DNA sequencing은 human genome과 같은 whole genomes의 시퀀싱이며, http://dnasequencing.com 에 따르면: “DNA를 찾는 것이 우리의 유전적 구성 정보를 구성하는 물질을 정확히 발견하는 것이었다면, DNA sequencing은 우리가 그 정보를 읽을 수 있게 하는 과정의 발견이다.” DNA sequencing의 주요 기능은 DNA 분자 안에 nucleotides의 정확한 순서를 찾는 것이다. 또한, DNA sequencing 은 bases—adenine (A), guanine (G), cytosine (C), and thymine (T) 의 순서를 결정하는데 사용된다.
주요 DNA sequencing의 도전은 다음과 같다.
- FASTQ files을 생성하는 시퀀싱 기술이 몇가지 있으며, DNA sequences의 길이는 각 시퀀싱 기술에 따라 다르다.
- 입력 데이터 (FASTQ data) 는 크다 (단일 DNA sequence sample은 900 GB).
- 단일의 강력한 서버를 가지고, 하나의 DNA sequence를 처리하고 변이를 추출하는데 80시간 정도까지 시간이 많이 소요된다.
- DNA sequencing에 관여하는 많은 알고리즘과 단계가 있어서, open source tools의 적절한 조합을 선택하는 것은 중요한 도전이다. 예를 들어 mapping/alignment algorithms 과 parameters이 있다.
- Scalability—즉, mapper와 reducer의 수를 최적화 하는 것—을 성취하기가 어렵다.
본 챕터에서는 DNA데이터를 FASTQ 파일로 받아 최종적으로 주어진 DNA 데이터 세트를 위한 변이를 가지는 VCF (variant call format)을 생성하는 MapReduce 프로그램의 집합으로서 DNA sequencing을 구현한다.

![Figure 18-1. High-level view of DNA sequencing](fig18-1.png)

DNA sequencing 파이프라인은 Figure 18-2에 있고 다음 스텝을 포함한다:
1. Input data validation: performing quality control on input data such as FASTQ files
2. Alignment: mapping short reads to the reference genome
3. Recalibration: visualizing and post-processing the alignment, including base quality recalibration
4. Variant detection: executing the SNP calling procedure along with filtering SNP candidates

![Figure 18-2. DNA sequencing pipeline](fig18-2.png)
There is plenty of data to analyze and apply DNA sequencing to, and there are lots of open source algorithms for completing the previous four steps. Note that your choice of these open source tools will significantly affect your final results, so it’s important that you understand them well.

## Input Data for DNA Sequencing
The most common format for DNA sequencing data is FASTQ, a text-based format for storing both a biological sequence and its quality scores. For a given FASTQ file, every four lines represent a single DNA sequence.

The general syntax of a FASTQ file is as follows:

    <fastq>:= <block>+
    
    <block>:=@<seqname>\n<seq>\n[<seqname>]\n<qual>\n
    
    <seqname>:= [A-Za-z0-9_.:-]+
    
    <seq>:= [A-Za-z\n\.~]+
    
    <qual>:= [!-~\n]+
    
And here is an example:
    @NCYC361-11a03.q1k bases 1 to 1576
    
    GCGTGCCCGAAAAAATGCTTTTGGAGCCGCGCGTGAAAT...
    
    +NCYC361-11a03.q1k bases 1 to 1576
    
    !)))))****(((***%%((((*(((+,**(((+**+,-...
    
FASTQ 데이터는 paired 나 nonpaired 임. paired 일 때, DNA 시퀀싱의 input은 한쌍의 파일이 될 것이다: left_fille.fastq과 right_file.fastq.
이제 DNA sequencing pipeline의 첫번째 스텝을 살펴보고 MapReduce가 어떻게 DNA sequencing 문제를 해결하는지 살펴보자.

## Input Data Validation
DNA sequencing pipeline의 첫번째 단계는 FASTQ 파일의 quality를 다음과 같은 간단한 오픈 소스를 통해 verify하여 validation하는 것.

- FastQValidator
- FastQC

## DNA Sequence Alignment

Sequence alignment는 두개 이상의 DNA 나 단백질 서열을 비교하는 것.
For global sequence alignment, consider the following example with two input sequences over the same alphabet:
- Sequence 1: GCGCATGGATTGAGCGA
- Sequence 2: TGCGCCATTGATGACCA

Our output is a possible alignment of the two sequences:
![](seq.png)

We can observe three elements in the possible alignment output:
- Perfect matches (in bold)
- Mismatches (underlined)
- Insertions and deletions (called indels, presented without formatting)

For the alignment phase, we will use MapReduce/Hadoop along with the following open source tools:
- Burrows-Wheeler Aligner (BWA), an efficient program that aligns relatively short nucleotide sequences against a long reference sequence such as the human genome (see http://bio-bwa.sourceforge.net/)
- Sequence Alignment/Map (SAM) tools, which provide various utilities for manipulating alignments in the SAM format, including sorting, merging, indexing, and generating alignments in a per-position format (see http://samtools.sour ceforge.net/)

We will be working with files in the BAM format, which is the binary format of a SAM file.

## MapReduce Algorithms for DNA Sequencing
일반적으로 단일 데이터 샘플의 DNA sequencing 은 파워가 좋은 단일 서버에서 70+ 시간 이상이 소요된다 (FASTQ 포맷으로 약 400–900 GB).

MapReduce 알고리즘의 목적은 수시간에 결과를 scalable하게 만드는데 있다.

alignment, recalibration, variant detection를 위한 대부분의 오픈 소스 도구 (BWA, SAMtools, GATK)는 MapReduce phase our map() 과 reduce() functions에서 linux 명령어 라인의 인터페이스를 가지고 있기 때문에, Linux shell script를 불러 적절한 매개변수를 제공한다.

이러한 shell script를 실행하기 위해, [Free‐Marker templating language](http://freemarker.org/)를 사용할 것이며 이는 적당한 쉘 스크립트를 만들기 위한 템플리트와 함께 자바 오브젝트와 데이터 구조를 결합할 것이다 (Figure 18-3).

하나의 DNA sequence를 다른 것과 구별하기 위해, 개별 분석을 위해 할당할 것이며 각 분석을 위해 “analysis ID”라고 불리는 고유의 GUID를 할당하고 이용할 것이다. (이는 입력과 출력 디렉토리를 유지하도록 도와줄 것이다).


![Figure 18-3. FreeMarker template engine](fig18-3.png)

(Example)
![](http://freemarker.org/images/overview.png)

MapReduce solution 은 3개의 단계로 이루어진다 (Figures 18-4, 18-5). 이는 본 챕터의 시작부분에서 서술된 파이프라인의 2-4번째 단계에 해당한다.

![Figure 18-4. MapReduce solution (steps 1 and 2)](fig18-4.png)

![Figure 18-5. MapReduce solution (step 3)](fig18-5.png)

세 단계의 MapReduce solution을 위한 DNA sequencing data flow 는 Figure 18-6과 같으며 각 단계에서 어떻게 데이터가 partition 되고 merge 되는지 보여준다.

![Figure 18-6. DNA sequencing data flow](fig18-6.png)

### Step 1: Alignment

![Figure 18-7. DNA sequencing alignment workflow](fig18-7.png)

alignment 단계가 시작하기 전에, DNA sequence FASTQ file을 8 백만 라인의 세트로 나뉜다고 하자 (또는 2백만 시퀀스). paired data에서 다음과 같이 나눈다:
```
left_file.fastq.0000 right_file.fastq.0000
    left_file.fastq.0001 right_file.fastq.0001
    left_file.fastq.0002 right_file.fastq.0002
    ...                  ...
```    
nonpaired data에서는 다음과 같이 나눈다:
```
    file.fastq.0000
    file.fastq.0001
    file.fastq.0002
    ...
```    
각 partition은 map() 함수를 통해 사용될 것이다. 파티션을 나누는 수는 Hadoop cluster의 크기에 의해 결정되어야 한다. 즉, 각기 4개의 mapper를 다루는 50개의 노드로 구성된 클러스트가 있다면 FASTQ 파일을 200개 partition으로 나누어야 한다. 예를 들어, 전체 입력 크기가 약 400 GB이면, 입력을 2 GB chunks로 자른다 (즉, 모든 mapper의 사용을 최대화호).

map() 함수는 BWA를 사용하여 하나의 chunk인 입력 파일을 읽어 BAM 형식으로 align된 파일을 생성할 것이다. alignment 가 완료되면, 모든 chromosomes을 뽑아내어 (1, 2, ..., 22, 23^2) MapReduce filesystem으로 저장한다 (Hadoop에서는 HDFS). 800 partitions이 있다면, chromosome 당 800 개의 파일을 생성할 것이며 (23 * 800 = 18,400 files) 23개의 reducers만이 있다 (chromosome 당 하나).

reducer는 특정 chromosome ID로 모든 chromosomes을 합칠 것이다 (merge and sort). 모든 chromosomes 1 은 chr1.bam라는 하나의 파일로 합쳐지고, 모든 chromosomes 2 은 chr2.bam라는 하나의 파일로 합쳐지는 것이 반복될 것이다. 그러고 나서 각 reducer는 recalibration을 위해 병합된 BAM 파일을 작은 파일로 합칠 것이다.

#### Mapper for the alignment phase
FASTQ format을 입력으로 받아서 partitioned chromosomes을 생성한다 (chr1, chr2, ..., chr22, chr23).

Example 18-1. Alignment phase: map() function

In [ ]:
1 /**
2 * @param key is a key generated by MapReduce framework
3 * @param value is a partitioned FASTQ file (may be 8M lines = 2M sequences)
4 */
5 map(key, value) {
6 // note: chr23 = concat(chrX, chrY, chrM)
7    alignedBAMFile = alignByBWA(value);
8    (chr1File, chr2File, ..., ch23File) = partitionByChromosome(alignedFile);
9 for(i=1,i<24;i++){
10       emit(chr<i>, chr<i>File);
11 }
12}

alignByBWA() 함수는 partitioned FASTQ file을 받아 alignment를 수행하고 최종적으로 chromosome별로 aligned file로 나뉜다. 이러한 동작 모두는 shell script template에 의해 수행된다.
Example 18-2. Alignment phase: nonpaired input

In [ ]:
1 #!/bin/bash
2 ...
3 export BWA=<bwa-install-dir>/bwa
4 export SAMTOOLS=<samtools-install-dir>/samtools
5 export BCFTOOLS=<bcftools-install-dir>/bcftools
6 export VCFUTILS=<bcftools-install-dir>/vcfutils.pl
7 export HADOOP_HOME=<hadoop-install-dir>
8 export HADOOP_CONF_DIR=<hadoop-install-dir>/conf
9 ...
10 # data directories
11 export TMP_HOME=<root-tmp-dir>/tmp
12 export BWA_INDEXES=<root-index-dir>/ref/bwa
13 ...
14 # define ref. genome
15 export REF=<root-reference-dir>/hg19.fasta
16
17 ### step 1: alignment
18 # the KEY uniquely identifies the input file
19 KEY={key}
20 # input_file
21 export INPUT_FILE=${input_file}
22 export ANALYSIS_ID=${analysis_id}
23 NUM_THREAD=3
24 cd $TMP_HOME
25 $BWA aln -t $NUM_THREAD $REF $INPUT_FILE > out.sai
26 $BWA samse -r $REF out.sai $INPUT_FILE | $SAMTOOLS view -Su -F 4 - | \ 27 $SAMTOOLS sort - aln.flt
28
29 # start indexing aln.flt.bam file
30 $SAMTOOLS index aln.flt.bam
31
32 # partition aligned data
33 for i in {1..22}
34 do
35 CHR=chr$i
36 $SAMTOOLS view -b -o $CHR.bam aln.flt.bam $CHR
37 output_file=/genome/dnaseq/output/$ANALYSIS_ID/$CHR/$KEY.$CHR.bam
38 $HADOOP_HOME/bin/hadoop fs -put $CHR.bam $output_file
39 done
40
41 # do the same thing for X, Y and M chromosomes
42 $SAMTOOLS view -b -o chr23.bam aln.flt.bam chrX chrY chrM
43 output_file=/genome/dnaseq/output/$ANALYSIS_ID/chr23/$KEY.chr23.bam
44 $HADOOP_HOME/bin/hadoop fs -put chr23.bam $output_file
45
46 exit 0

참고자료) hadoop fs 명령어 http://blog.acronym.co.kr/370
빅데이터 처리를 위한 MapReduce http://blog.acronym.co.kr/312
hadoop HDFS 명령어 http://pacino.tistory.com/entry/hadoop-hdfs-%EB%AA%85%EB%A0%B9%EC%96%B4
하둡의 명령어 http://yeiser.tistory.com/entry/2-%ED%95%98%EB%91%A1%EC%9D%98-%EB%AA%85%EB%A0%B9%EC%96%B4

paired input에는 25–27 line을 다음과 같이 수정한다.
Example 18-3. Alignment phase: paired input

In [ ]:
25 $BWA aln -t $NUM_THREAD $REF $INPUT_FILE_1 > out1.sai
26 $BWA aln -t $NUM_THREAD $REF $INPUT_FILE_2 > out2.sai
27 $BWA sampe -r $INFO_RG $REF out1.sai out2.sai $INPUT_FILE_1 $INPUT_FILE_2 | \ 
28 $SAMTOOLS view -Su -F 4 - | $SAMTOOLS sort - aln.flt

추가) 최근에는 더 성능이 우수한 BWA mem을 많이 사용한다. 또한 sort와 추가로 add or replace read groups를 picard를 통해 수행해준다.
이를 위해 다음과 같이 코드를 수정해준다.

In [ ]:
export PICARD=<samtools-install-dir>/picard.jar
export PROJECT=<PROJECT_NAME>
export PLATFORM=illumina
# 1. Actual Alignment. -I option to use illumina 1.3+ quailities. For the latest version, we don't need -I option. 
$BWA mem -t $NUM_THREAD -M $REF $INPUT_FILE_1 $INPUT_FILE_2 > $AlignedPath$SAMPLE.sam

# 2. Sort Sam
java -Xmx32g -jar $PICARD SortSam \
SO=coordinate \
INPUT=$AlignedPath$SAMPLE.sam \
OUTPUT=$AlignedPath$SAMPLE.bam \
VALIDATION_STRINGENCY=LENIENT \
CREATE_INDEX=true \
TMP_DIR=$AlignedPath/tmp


# 3. Add or replace read groups
java -Xmx32g -jar $PICARD AddOrReplaceReadGroups \
INPUT=$AlignedPath$SAMPLE.bam \
OUTPUT=$AlignedPath$SAMPLE.RGadded.bam \
SORT_ORDER=coordinate \
RGLB=${PROJECT} \
RGPL=${PLATFORM} \
RGPU=${PLATFORM} \
RGSM=${SAMPLE} \
CREATE_INDEX=true \
VALIDATION_STRINGENCY=LENIENT \
TMP_DIR=$AlignedPath/tmp

# 4. Remove samfile
rm $AlignedPath$SAMPLE$OPT.sam


#### Reducer for the alignment phase

alignment phase를 위해, 정확하게 23개의 reducer가 필요하다 (chromosome 당 하나의 reducer). reducer key 는 <chrID><;><analysisID> 와 같이 구성되어 있다. 이때 chromosome ID 23 은 chrM, chrX, chrY를 포함한다. 각 reducer 는 모든 aligned .bam 파일을 하나의 병합된 chr<i>.bam file 파일로 합친다:

    chr<i>.bam = merge the following files:
                 chr<i>.bam.0000
                 chr<i>.bam.0001
                 ...
                 chr<i>.bam.0437
                 ...
                 
모든 파일을 하나의 chr<i>.bam 파일로 합친후 we partition chr<i>.bam을 많은 작은 .bam file로 나눠서 step2의 recalibration mapper 로 만든다. partitioned file은 다음과 같이 될 것임:
chr<i>.bam.j (j = 1, 2, 3, ..., 100+)


Example 18-4. Alignment phase: reduce() function

In [ ]:
1 /**
2 * @param key is a <chrID><;><analysis_id>
3 * where chrID is in (1, 2, 3, ..., 23)
4 * @param value is ignored (not used)
5 */
6 reduce(key, value) {
7 DNASeq.mergeAllChromosomesAndPartition(key);
8}

DNASeq.mergeAllChromosomesAndPartition() 방법으로 특정한 chromosome의 모든 align된 .bam 파일을 병합하는 많은 양의 작업이 놓여 있다. 최종 merged file은 recalibration phase에 의해 더 진행되도록 나눠진다 (step 2).

Example 18-5. mergeAllChromosomesAndPartition() method

In [ ]:
1 /**
2 * reducerKey=<chrID>;<analysis_id>
3 * where chrID=1, 2, ..., 22, 23 (23 includes chrM, chrX, chrY)
4 */
5 public static void mergeAllChromosomesAndPartition(String reducerKey)
6 throws Exception {
7 // split the line: each line has two fields (fields are separated by ";")
8    String[] tokens = reducerKey.split(";");
9    String chrID = tokens[0];
10     String analysisID = tokens[1];
11  Map<String, String> templateMap = new HashMap<String, String>();
12     templateMap.put("chr_id", chrID);
13     templateMap.put("analysis_id", analysisID);
14  mergeAllChromosomesBamFiles(templateMap);
15 partitionSingleChromosomeBam(templateMap);
16}

mergeAllChromosomesAndPartition() 방법에서 볼 수 있는 것처럼, helper 방법인 mergeAllChromosomesBamFiles() 과 partitionSingleChromosomeBam() 모두 required Java objects 를 통과하고 reducers를 위한 shell scripts 을 위해 FreeMarker template engine 을 사용한다.

Example 18-6. mergeAllChromosomesBamFiles() method

In [ ]:
1 /**
2 * This method will merge the following files and create a single chr<i>.bam file
3 * where i is in {1, 2, ..., 23}:
4 *
5 * HDFS: /.../chr<i>/chr<i>.bam.0000
6 * HDFS: /.../chr<i>/chr<i>.bam.0001
7 * ...
8 * HDFS: /.../chr<i>/chr<i>.bam.0437
9 *
10 * Then merge all these (.0000, .0001, ..., .0437) files and save the result in
11 * /data/tmp/<analysis_id>/chr<i>/chr<i>.bam
12 *
13 * Once chr<i>.bam is created, then we partition it into small .bam files,
14 * which will be fed to RecalibrationDriver (step 2 of DNA sequencing)
15 *
16 */
17 public static void mergeAllChromosomesBamFiles(Map<String, String> templateMap)
18  throws Exception {
19  TemplateEngine.initTemplatEngine();
20  String templateFileName = <freemarker-template-file-as-a-bash-script>;
21  // create the actual script String chrID = templateMap.get("chr_id");from a template file
22  String chrID = templateMap.get("chr_id");
23  String analysisID = templateMap.get("analysis_id");
24  String scriptFileName = createScriptFileName(chrID, analysisID);
25  String logFileName = createLogFileName(chrID, analysisID);
26  File scriptFile = TemplateEngine.createDynamicContentAsFile(templateFileName,
27                                                            templateMap,
28                                                            scriptFileName);
29  if (scriptFile != null) {
30   ShellScriptUtil.callProcess(scriptFileName, logFileName);
31  }
32}

TemplateEngine.createDynamicContentAsFile() 방법은 마술과 같다 : 두 개의 input (templateFileName, templateMap)을 받아 scriptFile 이름을 생성한다. 기본적으로, 모든 parameter는 templateFileName으로 전달되고 새로운 shell script 가 reducer 대신 실행되도록 scriptFileName으로 생성된다.

ShellScriptUtil와 TemplateEngine는 두 개의 중요한 클래스이다. ShellScriptUtil.callProcess() 방법은 shell script file을 첫번째 매개변수로 받아 실행하고 두번째 매개변수로 모든 로그 파일을 생성하는 스크립트로부터 모든 로그를 생성한다. Logging은 비동기적이며 스크립트를 실행할때 즉시 로그 파일이 가능해진다.
TemplateEngine class는 template를 받아들이고 (as a text file with key holders) key-value pairs를 Java map 처럼 템플리트에서 모든 키가 value에 의해 대치되는 새로운 파일을 생성한다.

Example 18-7. TemplateEngine class

In [ ]:
1 import java.io.File;
2 import java.io.Writer;
3 import java.io.FileWriter;
4 import java.util.Map;
5 import java.util.concurrent.atomic.AtomicBoolean; 6 import freemarker.template.Template;
7 import freemarker.template.Configuration;
8 import freemarker.template.DefaultObjectWrapper; 9
10 /**
11 * This class uses FreeMarker (http://freemarker.sourceforge.net/).
12 * FreeMarker is a template engine, a generic tool to generate text
13 * output (anything from shell scripts to autogenerated source code)
14 * based on templates. It's a Java package, a class library for Java
15 * programmers. It's not an application for end users in itself, but
16 * something that programmers can embed into their products.
17 *
18 * @author Mahmoud Parsian
19 *
20 */
21 public class TemplateEngine{
22
23    // you usually do it only once in the whole application life cycle
24    private static Configuration TEMPLATE_CONFIGURATION = null;
25    private static AtomicBoolean initialized = new AtomicBoolean(false);
26
27    // the following template directories will be loaded from configuration file
28    private static String TEMPLATE_DIRECTORY = "/home/dnaseq/template";
29 
30    public static void init() throws Exception {
31         if (initialized.get()) {
32           // it is already initialized and returning...
33           return;
34         }
35         initConfiguration();
36         initialized.compareAndSet(false, true);
37    }
38
39    static {
40         if (!initialized.get()) {
41              try {
42                 init();
43              }            
44              catch(Exception e) {
45                 theLogger.error("TemplateEngine init failed at initialization.", e);
46              }
47         }
48    }
49
50   // this suppports a single template directory
51   private static void initConfiguration() throws Exception {
52      TEMPLATE_CONFIGURATION = new Configuration();
53      TEMPLATE_CONFIGURATION.setDirectoryForTemplateLoading(
54         new File(TEMPLATE_DIRECTORY));
55      TEMPLATE_CONFIGURATION.setObjectWrapper(new DefaultObjectWrapper());
56      TEMPLATE_CONFIGURATION.setWhitespaceStripping(true);
57      // if the following is set, then undefined keys will be set to ""
58      TEMPLATE_CONFIGURATION.setClassicCompatible(true);
59   }
60
61   public static File createDynamicContentAsFile(...){...}

TemplateEngine class에서 가장 중요한 방법은 createDynamicContentAs File()이며 key-value pairs으로 템플리트 파일을 수용하여 key holders에서 주어진 key로 대체한 새로운 파일을 생성한다.

In [ ]:
1 /**
2     * @param templateFile is a template filename such as script.sh.template
3     * @param keyValuePairs is a set of (K,V) pairs
4     * @param outputFileName is a generated filename from templateFile
5 */
6 public static File createDynamicContentAsFile(String templateFile, 
7                                               Map<String,String> keyValuePairs,
8                                               String outputFileName)
9   throws Exception { 
10  if ((templateFile == null) || (templateFile.length() == 0)) {
11        return null;
12  }
13
14  Writer writer = null;
15  try{
16    // create a template: example "cb_stage1.sh.template2"
17    Template template = TEMPLATE_CONFIGURATION.getTemplate(templateFile);
18    // merge data model with template
19    File outputFile = new File(outputFileName);
20    writer = new BufferedWriter(new FileWriter(outputFile));
21    template.process(keyValuePairs, writer);
22    writer.flush();
23    return outputFile;
24  }
25  finally {
26   if (writer != null) {
27     writer.close();
28   }
29  }
30 }
31}

## Step 2: Recalibration
Recalibration is the second phase of our MapReduce DNA sequencing pipeline.
In the recalibration step, each map() function will work on a specific aligned chromosome.
The mapper will perform duplicate marking, local realignment, and recalibration.

The goal of map() is to create a local recalibration table filled with covariates. These local covariates will be merged by the single reducer to create the final single global file (recalibration table) that will be used by the map() function of the third and final step of DNA sequencing, variant detection.

The recalibration MapReduce algorithm (data flow) is presented in Figure 18-8.

![Figure 18-8. DNA sequencing: recalibration](fig18-8.png)

```
<counter><;><partitioned-bam-file><;><ref_genome><;><analysis_id>
```

- <counter> is an autogenerated sequence of numbers 0000, 0001, 0002, ....
- <partitioned-bam-file> is a chunk of partitioned aligned file.
- <ref_genome> refers to hg18 or hg19.
- <analysis_id> is a GUID for DNA sequencing (to distinguish one analysis fromanother).

```
Here is some sample input:
      0001;chr07.bam.0001;hg19;208
      0002;chr07.bam.0002;hg19;208
      0003;chr07.bam.0003;hg19;208
      ...
```

In [ ]:
Example 18-9. Recalibration phase: map() function
1 // key is MR generated, ignored here
2 // value is: <counter><;><partitioned-bam-file><;><ref_genome><;><analysis_id> 3 map(key, value) {
4 // actual file location will be:
5 // /data/dnaseq/align/ANALYSIS_ID/merged.bam.<KEY>
6 Map<String, String> tokens =
7     DNASeq.tokenizeRecalibrationMapperInput(value);
8     String reducerKey = tokens.get("analysis_id");
9     DNASeq.recalibrationMapper(tokens);
10    emit(reducerKey, value);
11}
12
13 public static void recalibrationMapper(Map<String, String> templateMap) 
14 throws Exception {
15 TemplateEngine.init();
16 // create the actual script from a template file
17 String key = templateMap.get("key");
18 String analysisID = templateMap.get("analysis_id");
19 String scriptFileName = createScriptFileName("recalibration_mapper",
20                                                key,
21                                                analysisID);
22 String logFileName = createLogFileName("recalibration_mapper",
23                                                key,
24                                                analysisID);
25 File scriptFile = TemplateEngine.createDynamicContentAsFile( 
26       "recalibration_mapper.template",
27      templateMap,
28      scriptFileName);
29  if (scriptFile != null) {
30       ShellScriptUtil.callProcess(scriptFileName, logFileName);
31 }
32}    
    
Example 18-10. Recalibration phase: reduce() function
1 // key: analysisID
2 // values: ignored
3 reduce(key, Iterable<Object> values) { 
4 DNASeq.recalibrationReducer(key); 
5 emit(key, key);
6}

Example 18-11. recalibrationReducer() method

Example 18-12. Recalibration mapper template

Example 18-13. Recalibration reducer template



### Step 3: Variant Detection
Variant detection (also known as SNP calling) is the final phase of DNA sequencing. The goal of this step is to generate variants in VCF (variant call format; developed by the 1000 Genomes Project). The map() function will use the BAM file generated by the map() function of the recalibration step, and the final single “recalibration table” file. The map() function will use open source tools (such as GATK and SAMtools) to generate partial variants (which are raw BCF—binary call format—files). The reducer will concatenate (sort and merge) the raw BCF files to generate a single VCF file. Once the VCF file is created, it can be used by many analytical algorithms, such as allelic frequency (covered in Chapter 21), family analysis, and the Cochran-Armitage trend test.
Variant detection is the process of finding bases in the NGS (next-generation sequencing) data that differ from the reference genome, such as hg19 or hg38; these refer to the version of the human genome assembly and determine the version of the corresponding reference annotations (for details, see http://bit.ly/ build_36_1_genome).


#### Mapper for the variant detection phase
The mapper accepts a chunked “realigned .bam” file and performs the following transformations on it:
- Base quality recalibration
- Variant calling and filtering

The bulk of the work is done by the DNASeq.theVariantDetectionMapper() method, which accepts the required parameters and creates a proper shell script from a given template. Finally, it executes the shell script. The mapper for the variant detection phase is provided in Example 18-14.

In [ ]:
Example 18-14. Variant detection phase: map() function
    
Example 18-15. theVariantDetectionMapper() method

Example 18-16. Variant detection mapper template

